### Ziel dieses Kapitels: State-of-the-Art Model

Dir Techniken zu zeigen, mit denen du das Maximum aus deinem Datensatz rausholen 


In [1]:
from fastai.vision.all import *
path = untar_data(URLs.IMAGENETTE)

In [2]:
dblock = DataBlock(
    blocks=(ImageBlock(), CategoryBlock()),
    get_items=get_image_files,
    get_y=parent_label,  # Nimm den Ordnernamen als Label 
    item_tfms=Resize(224),  # Kleineres Bild
    batch_tfms=aug_transforms(size=128, min_scale=0.5)  # Weniger VRAM
)
dls = dblock.dataloaders(path, bs=4)

In [3]:
from fastai.callback.fp16 import *
learn = Learner(dls, xresnet18(), loss_func=CrossEntropyLossFlat(), metrics=accuracy).to_fp16()
learn.fit_one_cycle(5, 3e-3)


/home/lukelo-tshakulongo/Bankoko/φ-σ-Ω/Computer Science/Code/Mulandu/DLConcepts/venv/lib/python3.12/site-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/home/lukelo-tshakulongo/Bankoko/φ-σ-Ω/Computer Science/Code/Mulandu/DLConcepts/venv/lib/python3.12/site-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()


epoch,train_loss,valid_loss,accuracy,time
0,1.762589,1.657759,0.501494,02:16
1,1.282994,1.076227,0.668409,02:14
2,1.033945,1.002424,0.697909,02:21
3,0.775264,0.762086,0.768857,02:20
4,0.747868,0.611592,0.810680,02:21


### Normalisierung
+ Das Modell lernt besser, wenn die Werte "sauber verteilt" sind:
Durchschnitt = 0, Standardabweichung = 1

Das bedeutet: Wir berechnen Mittelwert (mean) und Standardabweichung (std) über alle Bilder (außer über die Farbkanäle: Rot, Grün, Blau).

+ Wenn du ein Modell weitergibst, solltest du auch die Normalisierungswerte mitliefern.
+ Wenn du ein fremdes Modell benutzt, finde raus, welche Werte es benutzt hat!

In [ ]:
# Diese Zahlen sind nicht nah an 0 und 1
x, y = dls.one_batch()
x.mean(dim=[0,2,3]), x.std(dim=[0,2,3])


(TensorImage([0.4792, 0.4991, 0.4694], device='cuda:0'),
 TensorImage([0.2466, 0.2396, 0.2567], device='cuda:0'))

In [ ]:
def get_dls(bs, size):
    dblock = DataBlock(
        blocks=(ImageBlock, CategoryBlock),
        get_items=get_image_files,
        get_y=parent_label,
        item_tfms=Resize(224),
        batch_tfms=[
            *aug_transforms(size=size, min_scale=0.5),
            Normalize.from_stats(*imagenet_stats)   # Normalisierung
        ])
    return dblock.dataloaders(path, bs=bs)


dls = get_dls(4, 128)


In [9]:
# Werte zwischen 0 und 1
x, y = dls.one_batch()
x.mean(dim=[0,2,3]), x.std(dim=[0,2,3])

(TensorImage([ 0.0151, -0.0655,  0.0232], device='cuda:0'),
 TensorImage([1.2722, 1.2716, 1.3045], device='cuda:0'))

In [10]:
learn = Learner(dls, xresnet18(), loss_func=CrossEntropyLossFlat(), metrics=accuracy).to_fp16()
learn.fit_one_cycle(5, 3e-3)

/home/lukelo-tshakulongo/Bankoko/φ-σ-Ω/Computer Science/Code/Mulandu/DLConcepts/venv/lib/python3.12/site-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/home/lukelo-tshakulongo/Bankoko/φ-σ-Ω/Computer Science/Code/Mulandu/DLConcepts/venv/lib/python3.12/site-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()


epoch,train_loss,valid_loss,accuracy,time
0,1.658547,1.583611,0.498880,02:15
1,1.327844,1.414874,0.625467,02:12
2,0.934927,0.772694,0.762509,02:15
3,0.893581,0.739198,0.780433,02:24
4,0.679526,0.644652,0.801344,02:22


### Progressive Resizing
+ Starte das Training mit kleinen Bildern → Viel schneller
+ Beende das Training mit großen Bildern → Höhere Genauigkeit
+ Die Größe der Bilder ist anfangs nicht so wichtig – das Netz erkennt trotzdem Strukturen!

In [ ]:
# Schritt 1: Mit kleinen Bildern anfangen (z. B. 128×128)
dls = get_dls(64, 128)
learn = Learner(dls, xresnet18(), loss_func=CrossEntropyLossFlat(), metrics=accuracy).to_fp16()
learn.fit_one_cycle(5, 3e-3)


epoch,train_loss,valid_loss,accuracy,time
0,1.732646,1.778171,0.424944,00:26
1,1.228281,1.441292,0.529500,00:24
2,0.951116,0.880464,0.715459,00:25
3,0.769573,0.720444,0.768111,00:25
4,0.654340,0.621007,0.797237,00:25


In [12]:
# Schritt 2: Bildgröße erhöhen & feinjustieren (z. B. 224×224)
# Finale Genauigkeit steigt! Und: Anfangstraining war viel schneller durch die kleinen Bilder.
# Wie oft darf man Bildgröße steigern?
    # So oft du willst! Z. B. von 128 → 224 → 288 → 320 …

learn.dls = get_dls(64, 224)
learn.fine_tune(5, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,0.679435,1.082562,0.682226,00:34


epoch,train_loss,valid_loss,accuracy,time
0,0.594649,0.808173,0.741598,00:37
1,0.579121,0.985817,0.703510,00:38
2,0.515553,0.598878,0.812173,00:41
3,0.463943,0.610183,0.802091,00:40
4,0.442395,0.592477,0.812173,00:41


### Test Time Augmentation
Beim Testen oder Validieren mehrere leicht veränderte Versionen eines Bildes erzeugen → Modell mehrfach vorhersagen lassen → Ergebnisse mitteln oder den besten nehmen.

Dadurch lernt das Modell allgemeiner, also robuster.
Aber: Beim Validieren (also beim Testen, wie gut das Modell ist), nimmt fastai nur den mittleren Teil des Bildes (Center Crop).

Stell dir vor:

*   Das Bild ist länglich, aber der wichtige Teil (z. B. eine Nase 🐶) ist am Rand

*   Durch den mittigen Zuschnitt wird dieser Teil abgeschnitten!


Statt nur 1 Zuschnitt beim Testen, nimm z. B. 5 Varianten eines Bildes:

*   Andere Ausschnitte (nicht nur Mitte)

*  Leichte Drehungen, Helligkeit, Spiegelung usw.


In [13]:
preds, targs = learn.tta()
accuracy(preds, targs).item()

0.8749066591262817

### Was ist Mixup?

Genauigkeit stark verbessern, besonders wenn du:

*   wenig Trainingsdaten hast

*   kein vortrainiertes Modell nutzt, das deinem Datensatz ähnlich ist


Für jedes Bild im Training:

*    Zufällig ein zweites Bild aus dem Datensatz auswählen

*    Zufälliges Gewicht t wählen (z. B. 0.3 oder 0.7)

*    Erstelle neues Bild = t * Bild1 + (1 - t) * Bild2

*    Erstelle neues Label = t * Label1 + (1 - t) * Label2


# Mixup in FastAI – Erklärung und Anwendung

## 1. Was ist Mixup?

**Mixup** ist eine Datenaugmentierungstechnik, bei der zwei Trainingsbeispiele (Bild + Label) linear miteinander vermischt werden.  
Das Modell wird dadurch robuster, weil es lernt, mit mehrdeutigen (unsicheren) Situationen umzugehen.

---

## 2. Pseudocode-Beispiel

```python
image2, target2 = dataset[random_index]
t = random_float(0.5, 1.0)

new_image = t * image1 + (1 - t) * image2
new_target = t * target1 + (1 - t) * target2
```

> **Wichtig:** Das funktioniert nur, wenn deine Labels **One-Hot-codiert** sind (z. B. `[0, 0, 1, 0, …]`).

---

## 3. Visuelles Beispiel

Angenommen:

- **Bild 1:** Kirche  
  → Label = `[0, 0, 1, 0, ..., 0]`

- **Bild 2:** Tankstelle  
  → Label = `[0, 0, 0, 0, ..., 1]`

Wenn wir mit `t = 0.3` mischen:

- **Bild =** 30 % Kirche + 70 % Tankstelle  
- **Label =** `[0, 0, 0.3, 0, ..., 0.7]`

> Das Modell lernt: **"Dieses Bild ist ein bisschen beides!"**

Ein gemischtes Bild erzeugt eine neue Herausforderung für das Modell – es muss **anteilige Klassen** erkennen.

---

## 4. Anwendung in FastAI

```python
learn = Learner(
    dls, 
    xresnet50(), 
    loss_func=CrossEntropyLossFlat(), 
    metrics=accuracy, 
    cbs=Mixup
)

learn.fit_one_cycle(5, 3e-3)
```

- Das `cbs=Mixup` aktiviert Mixup als **Callback**, der bei jedem Batch automatisch zwei Bilder und ihre Labels mischt.

---

## 5. Was passiert beim Training?

- Das Modell sieht **gemischte Bilder** anstelle von klaren Einzelbeispielen.
- Es lernt, **mehrdeutige Labels** wie `[0.3, 0.7]` zu verstehen.
- Dadurch wird das Modell robuster und **vermeidet Overfitting**.
- Jedes Bild ist im Grunde **neu erzeugt** – also **mehr Varianz**.

---

## 6. Vorteile

- **Reduziert Overfitting**
- **Erhöht Generalisierbarkeit**
- **Ideal für lange Trainings** (z. B. > 80 Epochen)
- Funktioniert auch bei **Textdaten** oder **Aktivierungen im Modell**

---

## 7. Nachteile

- **Training ist schwieriger**, weil das Modell mehrdeutige Daten lernen muss
- **Benötigt mehr Epochen** bis zur Konvergenz
- **Labels sind "verfälscht"**, ohne dass das Modell es weiß
- Die Stärke der Mischung ist zufällig – man hat **wenig Kontrolle**, es sei denn, man passt `t` explizit an

---

## 8. Mathematischer Vorteil

- Klassische Labels sind 0 oder 1  
  → Das Modell versucht ständig, mit Softmax diese Werte zu erreichen, was unrealistisch ist

- **Mixup-Labels liegen zwischen 0 und 1** (z. B. `0.3` und `0.7`)  
  → Dadurch kann das Modell realistischere Wahrscheinlichkeiten lernen und **überschießende Aktivierungen vermeiden**

---

## 9. Wann Mixup einsetzen?

| Trainingsdauer      | Mixup sinnvoll?       |
|---------------------|------------------------|
| Kurzes Training     | ❌ Nicht hilfreich     |
| Langes Training     | ✅ Sehr hilfreich      |


# Was ist Label Smoothing?

„Das richtige Label ist 100 % korrekt, alle anderen sind 100 % falsch.“

[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]


Label Smoothing zeigt seine Wirkung erst bei längeren Trainingsläufen (z. B. 30, 50, 80 Epochen)


In [17]:
# Das ist alles – du brauchst nur eine andere Loss-Funktion!
dls = get_dls(64, 128)


learn = Learner(dls, xresnet18(), 
                loss_func=LabelSmoothingCrossEntropy(),  # Hier
                metrics=accuracy).to_fp16()

learn.fit_one_cycle(5, 3e-3)


/home/lukelo-tshakulongo/Bankoko/φ-σ-Ω/Computer Science/Code/Mulandu/DLConcepts/venv/lib/python3.12/site-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/home/lukelo-tshakulongo/Bankoko/φ-σ-Ω/Computer Science/Code/Mulandu/DLConcepts/venv/lib/python3.12/site-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()


epoch,train_loss,valid_loss,accuracy,time
0,2.516923,2.550240,0.479089,00:26
1,2.054281,2.104333,0.613518,00:29
2,1.846828,1.971688,0.670276,00:28
3,1.693976,1.610719,0.804705,00:28
4,1.592195,1.562520,0.815534,00:27


## Problem: Zu viel Sicherheit beim Lernen

Wenn wir ein Modell klassisch trainieren, versucht es:

- Für das **korrekte Label**: Ausgabe = `1`
- Für **alle anderen Klassen**: Ausgabe = `0`

### Aber in der Realität:

- **Softmax/Sigmoid** können **niemals exakt** `1` oder `0` zurückgeben.
- Das Modell versucht dennoch, diese Extreme zu erreichen (z. B. `0.9999`).
- Das führt zu:

  - Overfitting
  - Übertriebene Selbstsicherheit
  - Schlechte Generalisierung beim Testen

---

## Beispiel: Fehlerhafte Labels

Einige Bilder im Datensatz könnten **falsch beschriftet** sein, z. B.:

- Ein Bild zeigt zwei Tierarten
- Das Label ist schlicht falsch

Wenn du das Modell zwingst, immer "1 oder 0" zu lernen, wird es bei solchen Fehlern verwirrt.

---

## Lösung: Label Smoothing

Statt:

```
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
```

nutzen wir z. B.:

```
[0.01, 0.01, 0.01, 0.91, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]
```

Das sagt dem Modell:

> „Ich bin zu 91 % sicher, dass es Klasse 3 ist – aber nicht absolut sicher.“

### Das bringt:

- Robusteres Modell
- Bessere Generalisierung
- Weniger anfällig für fehlerhafte Daten

---

## Technische Erklärung

Angenommen:

- Anzahl der Klassen `N = 10`
- Unsicherheitsfaktor `ε = 0.1`

Dann:

- Jeder "falsche" Wert: `ε / N = 0.01`
- Der "richtige" Wert: `1 - ε + (ε / N) = 0.91`

> Die Summe der Werte bleibt **exakt 1** – mathematisch korrekt

---

## Erkenntnisse aus dem Google-Paper (Szegedy et al.)

1. **Zielwert 1 ist nicht erreichbar**

   - Softmax erreicht nie exakt `1`
   - Trotzdem wird das Modell dahin „gepusht“ → führt zu extremen Ausgaben

2. **Übermäßige Sicherheit schadet**

   - Modell glaubt zu stark an sich selbst → Generalisiert schlechter

3. **Unflexible Gradienten**

   - Die Differenz zwischen der "wichtigsten Klasse" und den anderen ist zu groß
   - Dadurch werden Gradienten für kleinere Klassen zu klein
   - → Schwache Anpassung bei Transfer Learning
